In [3]:
import sys
import gc
import re
import json
import _pickle as p
import pandas as pd
import numpy as np
from importlib import reload
from IPython.display import display, HTML

sys.path.append('..')
sys.path.append('../lib')
import stats as S
S = reload(S)

pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
display(HTML("<style>.container{width: 100%}</style>"))

/home/aynur/Installed/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


In [5]:
lemm = S.lemm_h
def freq_dict(text, suff='ph'):
    txt_lemm = lemm(txt)
    kwds = S.keywords(txt_lemm)
    word_cnt = len(kwds)
    word_cnt_log_inv = 1 / np.log2(word_cnt)
    kwds_sorted = sorted(kwds.items(), key=lambda x: -x[1])
    kwds_inverted = {i: word_cnt_log_inv/v for i, v in kwds.items()}
    v = kwds_inverted.values()
    with open(f'word_importance.{suff}.json', 'w') as f:
        f.write(json.dumps(kwds_inverted))
    print(f"Stats: words count: {word_cnt}, log_inv = {word_cnt_log_inv}; min = {min(v)}, max = {max(v)}")
    return kwds_inverted

In [ ]:
articles = pd.read_csv('medical_site.csv')
pharma = pd.read_csv('pharma.csv')
pharma.drop(['web-scraper-order', 'web-scraper-start-url', 'Page', 'Page-href', 'PharmaLink', 'AnalogsLinks-href', 'Lots', 'Instruction'], axis=1, inplace=True)

txt = "\n".join(articles['ArticleText'].map(str)) + "\n" + "\n".join(pharma['PharmacologicAction'].map(str))

kwds_inverted_ph = freq_dict(txt, suff='ph')

Ph = pharma.loc[pharma.loc[:, 'PharmacologicAction'].notnull()]
Ph.loc[:, 'Lemm'] = Ph.loc[:, 'PharmacologicAction'].apply(lemm)
Ph.loc[:, 'Kw'] = Ph.loc[:, 'Lemm'].apply(lambda x: {i: 1 for i, v in S.keywords(x).items()})
Ph.loc[:, 'KwMult'] = Ph.loc[:, 'Kw'].apply(lambda x: S.multiply2stats(kwds_inverted_ph, x, max_dist=0))

Ph.drop(['Lemm', 'Kw'], axis=1, inplace=True)
Ph.to_csv('pharma_kw.csv')

gc.collect()

Stats: words count: 21474, log_inv = 0.06949123844910178; min = 3.5416766958412813e-06, max = 0.06949123844910178


/home/aynur/Installed/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/aynur/Installed/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [ ]:
pattern = re.compile('[^\w\d\s]+') 
Sdict = pd.read_csv('med_terms.csv')
Sdict.drop(['web-scraper-order', 'web-scraper-start-url', 'Pages', 'Pages-href'], axis=1, inplace=True)

txt = "\n".join(Sdict['Definitions'].map(str))
kwds_inverted_def = freq_dict(txt, suff='def')

Sdict.loc[:, 'Term'] = Sdict.loc[:, 'Definitions'].apply(lambda x: pattern.split(x, 1)[0])
Sdict.loc[:, 'Definitions'] = Sdict.loc[:, 'Definitions'].apply(lambda x: pattern.split(x, 1)[1])
Sdict[['TermLemm', 'DefLemm']] = Sdict.loc[:, ['Term', 'Definitions']].applymap(S.lemm_m)
Sdict.loc[:, 'DefKw'] = Sdict.loc[:, 'DefLemm'].apply(lambda x: S.keywords(x))
# Sdict.loc[:, 'TermKw'] = Sdict.loc[:, 'TermLemm'].apply(lambda x: S.keywords(x))
Sdict.loc[:, 'KwMult'] = Sdict.loc[:, 'DefKw'].apply(lambda x: S.multiply2stats(kwds_inverted_def, x, max_dist=0))
# Sdict.loc[:, 'DefKwFilt'] = Sdict.loc[:, 'DefKwMult'].apply(lambda x: )
Sdict.drop(['DefKw', 'DefLemm'], axis=1, inplace=True)
Sdict.to_csv('med_terms_kw.csv')

gc.collect()

In [ ]:
Pdict = pd.read_csv('medical_profs.csv')
Pdict.drop(['web-scraper-order', 'web-scraper-start-url', 'ProfLinks-href'], axis=1, inplace=True)

txt = "\n".join(Pdict['DescriptionText'].map(str))
kwds_inverted_prof = freq_dict(txt, suff='prof')

Pdict[['ProfLemm', 'DescLemm']] = Pdict.loc[:, ['ProfLinks', 'DescriptionText']].applymap(S.lemm_m)
Pdict.loc[:, 'DescKw'] = Pdict.loc[:, 'DescLemm'].apply(lambda x: S.keywords(x))
Pdict.loc[:, 'KwMult'] = Pdict.loc[:, 'DescKw'].apply(lambda x: S.multiply2stats(kwds_inverted_prof, x, max_dist=0))
Pdict.drop(['DescKw', 'DescLemm', 'ProfLemm'], axis=1, inplace=True)
Pdict.drop([1, 3, 4, 5, 11, 16, 26, 27, 33], axis=0, inplace=True)
Pdict.to_csv('medical_profs_kw.csv')

In [ ]:
for ds, THRESHOLD, fname_basis in [
    (Ph, 1e-2, 'pharma_kw_thr'),
    (Pdict, 1e-2, 'medical_profs_kw_thr'),
    (Sdict, 1e-2, 'med_terms_kw_thr'),
]:
    ds.loc[:, 'KwFilt'] = ds.loc[:, 'KwMult'].map(lambda x: {i: v for i, v in x.items() if v > max(x.values())*THRESHOLD})
    ds.loc[:, 'FSum'] = ds.loc[:, 'KwFilt'].map(lambda x: sum(x.values()))
    ds.loc[:, 'KwNorm'] = ds.apply(lambda x: {i: v/x['FSum'] for i, v in x['KwFilt'].items()}, axis=1)
    ds.drop(['KwFilt', 'FSum'], axis=1, inplace=True)
    ds.to_csv(f'{fname_basis}{THRESHOLD:.6f}.csv')

In [ ]:
display(Ph.head())
display(Sdict.head())
display(Pdict.head())